In [1]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output


In [ ]:
read_cc = True and False
read_tb = True and False

In [ ]:
cc = CasosConfirmados()
if read_cc:
    cc.read_excel()
    cc.save()
else:
    cc.load()

In [ ]:
casosc = cc.get_casos()
casosc.loc[casosc['obito']!='SIM','obito'] = 'NAO'
print(f"casosc len: {len(casosc)}\n")
casosc.groupby('obito')[['sexo']].count()

In [ ]:
tb_pacientes = TbPacientes()
if read_tb:
    tb_pacientes.update()

casost = tb_pacientes.get()
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
casost.groupby('obito')[['sexo']].count()

In [ ]:
casost.loc[~(
    casost['hash'].isin(casosc['hash']) |
    casost['hash'].isin(casosc['hash_less']) |
    casost['hash'].isin(casosc['hash_more']) |
    casost['hash_atend'].isin(casosc['hash_atend']) |
    casost['hash_less_atend'].isin(casosc['hash_atend']) |
    casost['hash_more_atend'].isin(casosc['hash_atend']) |
    casost['hash_diag'].isin(casosc['hash_diag'])
)]

In [ ]:
del casosc['identificacao']

In [ ]:
tb_cols = ['identificacao','ibge_atend_pr','dt_com_obito','status','dt_com_recuperado','hash_less_atend','hash_more_atend']

casosct = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash'].isin(casost['hash']))],casost[['hash']+tb_cols], how='left', on='hash')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct =  casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash'])],casost[['hash']+ tb_cols], how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_less'])],casost[['hash_less']+ tb_cols].rename(columns={'hash_less':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_more'])],casost[['hash_more']+ tb_cols].rename(columns={'hash_more':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

In [ ]:
casosct = casosct.append(pd.merge(casosc,casost[['hash_diag']+ tb_cols], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))
casosct.loc[casosct['identificacao'].isna()]

In [ ]:
casosct['identificacao'] = casosct['identificacao'].astype(int)
casost.loc[~casost['identificacao'].isin(casosct['identificacao'])]

In [ ]:
id_duplicado = casosct.loc[casosct.duplicated('identificacao',keep=False)].sort_values('hash_diag')
ordem_duplicado = casosct.loc[casosct.duplicated('ordem',keep=False)].sort_values('hash_diag')
print(len(id_duplicado),len(ordem_duplicado))
all(id_duplicado['hash_diag'].values==ordem_duplicado['hash_diag'].values)

In [ ]:
# casosct.loc[casosct.duplicated('identificacao',keep='last')]
casosct.loc[casosct['comunicacao']==pd.to_datetime('2020-03-11'),'comunicacao'] = pd.to_datetime('2020-03-12')
id_duplicado.loc[~id_duplicado['hash_diag'].isin(ordem_duplicado['hash_diag'])]

In [ ]:
print(len(casosct))
casosct = casosct.drop_duplicates('identificacao',keep='last')
print('casos:',len(casosct))
print('obitos:',len(casosct.loc[casosct['obito']=='SIM']))

In [ ]:
try:
    casosct = casosct_bkp.copy()
except:
    casosct_bkp = casosct.copy()
    
# casosct = casosct.sort_values(['comunicacao','nome','idade']).copy()
casosct = casosct.sort_values(['ordem']).copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito'].notna(),'data_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito'].notna(),'dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_obito']

casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.groupby('evolucao')[['sexo']].count()

In [ ]:
casosc = cc.get_casos()
casosct.loc[~casosct['ordem'].isin(casosc['ordem'])]

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)

In [ ]:
casosct['uf_resid'] = 'PR'
casosct.loc[casosct['ibge7']==9999999,'uf_resid'] = casosct.loc[casosct['ibge7']==9999999,'mun_resid'].apply(lambda x: x.split('/')[-1])
casosct.loc[casosct['ibge7']==9999999,'mun_resid'] = casosct.loc[casosct['ibge7']==9999999,'mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casosct.loc[casosct['ibge7']==9999999].drop(columns=['ibge7']),municipios[['ibge7','mun_resid','uf']].rename(columns={'uf':'uf_resid'}), on=['mun_resid','uf_resid'], how='left')
# casos_fora.loc[casos_fora['ibge7'].isna(),'excluir'] = 'SIM'
casos_fora.loc[casos_fora['ibge7'].isna(),'ibge7'] = 9999999

In [ ]:
casos_fora = casos_fora.set_index('identificacao')
casosct = casosct.set_index('identificacao')
casosct.update(casos_fora)

casosct = casosct.reset_index()

casosct['ibge7'] = casosct['ibge7'].apply(int)
casosct['idade'] = casosct['idade'].astype(int)

casosct.groupby(['evolucao'])[['sexo']].count()

In [ ]:
#####
## novos casos confirmados
#

casosct = casosct.sort_values(['comunicacao','nome'])
casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge7'].apply(lambda x: int(str(x)[:6]))
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(lambda x: int(str(x)[:6]))

casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna(), 'hash_obito'] = casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna()].apply(
        lambda row: normalize_hash(row['nome']) + date_hash(row['data_com_evolucao']), axis=1
)

In [ ]:
casosct['hash'] = (casosct['nome'].apply(normalize_hash) +
                  casosct['idade'].astype(str) +
                  casosct['ibge_resid'].astype(str)  )

casosct['hash_less'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x - 1)) +
                        casosct['ibge_resid'].astype(str)  )

casosct['hash_more'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x + 1)) +
                        casosct['ibge_resid'].astype(str)  )

casosct['hash_diag'] = (casosct['nome'].apply(normalize_hash) + casosct['dt_diag'].apply(date_hash))

casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]#,'hash_atend','hash_less_atend','hash_more_atend','hash_diag','hash_obito']]
# casosct = casosct.set_index('identificacao')
casosct.groupby('evolucao')[['sexo']].count()

In [4]:
casosct = pd.read_pickle('casosct_22_05_2021.pkl')[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]
casosct.loc[casosct['comunicacao']=='23/05/2021','comunicacao'] = '22/05/2021'

In [5]:
novos_casos = pd.read_excel(join(default_input,'novos_casos.xlsx')).rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'exame': 'laboratorio',
    'data_liberacao':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao',
    'uf': 'uf_resid'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2
novos_casos.loc[~novos_casos['evolucao'].isin([1,2,3]),'evolucao'] = 3

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['identificacao'] = list(range(casosct.iloc[-1,0] + 1,casosct.iloc[-1,0] + 1 + len(novos_casos)))
casosct = casosct.append(novos_casos)
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1.0,730699
2.0,25485
3.0,304499


In [6]:
novos_obitos = pd.read_excel(join(default_input,'novos_obitos.xlsx')).rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2

In [7]:
novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')
casosct = casosct.set_index('hash')
casosct.update(novos_obitos)
casosct.reset_index(inplace=True)
casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]

print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

1060683


,sexo
evolucao,
1.0,730695
2.0,25506
3.0,304482


In [8]:
casosct.loc[casosct['laboratorio']==0,'laboratorio'] = 'NAO INFORMADO'

In [9]:
casosct.head()

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_diag
0,8838,-1.0,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12 00:00:00,NaT,1.0,NaT,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1.0,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-07 00:00:00,1.0,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1.0,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-08 00:00:00,1.0,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,19775,-1.0,PR,410690,410690,LUIZ ANTONIO CAXAMBU FREITAS,M,54,GENOPRIMER,2020-03-11,2020-03-12 00:00:00,2020-03-09 00:00:00,1.0,NaT,NaT,LUIZANTONIOCAXAMBUFREITAS54410690,LUIZANTONIOCAXAMBUFREITAS53410690,LUIZANTONIOCAXAMBUFREITAS55410690,LUIZANTONIOCAXAMBUFREITAS11032020
4,22343,-1.0,PR,410690,410690,MARIANA CAXAMBU FREITAS,F,25,GENOPRIMER,2020-03-11,2020-03-12 00:00:00,2020-03-01 00:00:00,1.0,NaT,NaT,MARIANACAXAMBUFREITAS25410690,MARIANACAXAMBUFREITAS24410690,MARIANACAXAMBUFREITAS26410690,MARIANACAXAMBUFREITAS11032020


In [10]:
casosct.tail()

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_diag
1060678,1145878,3805442.0,PR,410940,410940,YURI GABRIEL MENDES DE OLIVEIRA,M,20,"COVID-19, Biologia Molecular",2021-05-23,23/05/2021,NaN,3.0,NaT,NaT,YURIGABRIELMENDESDEOLIVEIRA20410940,YURIGABRIELMENDESDEOLIVEIRA19410940,YURIGABRIELMENDESDEOLIVEIRA21410940,NaN
1060679,1145879,3712006.0,PR,410940,410940,ZELIA DE FATIMA REJEI,F,50,"COVID-19, Biologia Molecular",2021-05-23,23/05/2021,15/05/2021,3.0,NaT,NaT,ZELIADEFATIMAREJEI50410940,ZELIADEFATIMAREJEI49410940,ZELIADEFATIMAREJEI51410940,NaN
1060680,1145880,3741044.0,PR,410660,410660,ZENAIDE NOGUEIRA,F,50,"COVID-19, Biologia Molecular",2021-05-21,23/05/2021,15/05/2021,3.0,NaT,NaT,ZENAIDENOGUEIRA50410660,ZENAIDENOGUEIRA49410660,ZENAIDENOGUEIRA51410660,NaN
1060681,1145881,3806108.0,PR,410480,410480,ZENI ALVES DE MORAES,F,52,Teste Rápido Antígeno (Imunocromatografia),2021-05-12,23/05/2021,07/05/2021,1.0,2021-05-22,NaT,ZENIALVESDEMORAES52410480,ZENIALVESDEMORAES51410480,ZENIALVESDEMORAES53410480,NaN
1060682,1145882,3769696.0,PR,411180,411180,ZILDA APARECIDA PRADO DE OLIVEIRA,F,43,"COVID-19, Biologia Molecular",2021-05-22,23/05/2021,15/05/2021,3.0,NaT,NaT,ZILDAAPARECIDAPRADODEOLIVEIRA43411180,ZILDAAPARECIDAPRADODEOLIVEIRA42411180,ZILDAAPARECIDAPRADODEOLIVEIRA44411180,NaN


In [11]:
casosct.to_pickle('casosct_23_05_2021.pkl')

In [33]:
municipios = Municipios()

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casosct = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casosct['municipio_pr'] = casosct['municipio']
casosct.loc[casosct['uf_resid']!='PR','municipio_pr'] = 'Fora'

casosct.loc[casosct['uf_resid']!='PR','ibge7'] = 9999999

casosct['uf_pr'] = 'PR'
casosct.loc[casosct['uf_resid']!='PR','uf_pr'] = 'Fora'

In [34]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

,PR,Fora,GERAL
CASOS,1.054.893,5.790,1.060.683
OBITOS,25.362,144,25.506
RECUPERADOS,728.523,2.172,730.695
ATIVOS,301.008,3.474,304.482


In [35]:
total_macros = casosct.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
Leste,490.279,12.922,327.498,149.859
Oeste,217.680,4.222,183.238,30.220
Noroeste,169.919,3.618,125.116,41.185
Norte,177.015,4.600,92.671,79.744
Fora,5.790,144,2.172,3.474


In [36]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
rs,regional,,,,
1,Paranaguá,33.375,799,17.504,15.072
2,Metropolitana,315.576,9.109,222.214,84.253
3,Ponta Grossa,61.385,1.455,40.433,19.497
4,Irati,14.333,243,10.140,3.950
5,Guarapuava,32.739,658,18.926,13.155
6,União da Vitória,10.566,208,7.676,2.682
7,Pato Branco,28.248,629,23.569,4.050
8,Francisco Beltrão,38.096,638,30.223,7.235
9,Foz do Iguaçu,54.340,1.104,42.116,11.120


In [37]:
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

evolucao                                         CASOS  OBITOS  RECUPERADOS  \
macro rs regional  ibge7   municipio_pr                                       
Leste 1  Paranaguá 4101200 Antonina              2.215      56        1.987   
                   4109500 Guaraqueçaba            593      14          477   
                   4109609 Guaratuba             4.225     148        3.731   
                   4115705 Matinhos              2.670      85        2.276   
                   4116208 Morretes              1.850      49        1.472   
...                                                ...     ...          ...   
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí      258       8          224   
                   4122651 Rosário do Ivaí         376       2          336   
                   4123857 Santa Maria do Oeste    578      22          470   
                   4125001 São João do Ivaí        785      16          674   
Fora  99 Fora      9999999 Fora                  5.790     144        2.172   

evolucao                                         ATIVOS  
macro rs regional  ibge7   municipio_pr                  
Leste 1  Paranaguá 4101200 Antonina                 172  
                   4109500 Guaraqueçaba             102  
                   4109609 Guaratuba                346  
                   4115705 Matinhos                 309  
                   4116208 Morretes                 329  
...                                                 ...  
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí        26  
                   4122651 Rosário do Ivaí           38  
                   4123857 Santa Maria do Oeste      86  
                   4125001 São João do Ivaí          95  
Fora  99 Fora      9999999 Fora                   3.474  

[400 rows x 4 columns]

In [38]:
casosct['regional'] = casosct['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casosct['regional']
casosct.columns

Index(['identificacao', 'id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend',
       'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is',
       'evolucao', 'data_evolucao', 'data_com_evolucao', 'hash', 'hash_less',
       'hash_more', 'hash_diag', 'ibge7', 'rs', 'regional', 'macro',
       'municipio', 'uf', 'populacao', 'estado', 'municipio_pr', 'uf_pr'],
      dtype='object')

In [18]:
casos_confirmados = casosct[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [19]:
exit()

SystemExit: 

/home/vinnylg/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
writer = pd.ExcelWriter("casos_confirmados.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for macro in casos_confirmados['macro'].unique():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==macro].copy()
    casos_macro.to_excel(writer,macros_idx_labels[macro])
    fit_cols(writer,casos_macro,macros_idx_labels[macro])


writer.save()

In [ ]:
writer = pd.ExcelWriter("obitos_confirmados.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for macro in obitos_confirmados['macro'].unique():
    obitos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==macro].copy()
    obitos_macro.to_excel(writer,macros_idx_labels[macro])
    fit_cols(writer,obitos_macro,macros_idx_labels[macro])


writer.save()

In [ ]:
writer = pd.ExcelWriter("casos_confirmados_regionais.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in casos_confirmados['regional'].unique():
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [ ]:
writer = pd.ExcelWriter("obitos_confirmados_regionais.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in obitos_confirmados['regional'].unique():
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()